In [1]:
from kalmanfilter import KalmanFilter
from datapoint import DataPoint
from fusionekf import FusionEKF
from tools import get_RMSE, cartesian_to_polar
from helpers import parse_data, get_state_estimations, print_EKF_data
import numpy as np

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [2]:
lidar_R = np.matrix([[0.01, 0], 
                     [0, 0.01]])

radar_R = np.matrix([[0.01, 0, 0], 
                     [0, 1.0e-6, 0], 
                     [0, 0, 0.01]])

lidar_H = np.matrix([[1, 0, 0, 0],
                     [0, 1, 0, 0]])

P = np.matrix([[1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 1000, 0], 
               [0, 0, 0, 1000]])

Q = np.matrix(np.zeros([4, 4]))
F = np.matrix(np.eye(4))

d = {
  'number_of_states': 4, 
  'initial_process_matrix': P,
  'radar_covariance_matrix': radar_R,
  'lidar_covariance_matrix': lidar_R, 
  'lidar_transition_matrix': lidar_H,
  'inital_state_transition_matrix': F,
  'initial_noise_matrix': Q, 
  'acceleration_noise_x': 5, 
  'acceleration_noise_y': 5
}

EKF = FusionEKF(d)

In [3]:
all_sensor_data, all_ground_truths = parse_data("data/data-1.txt")
all_state_estimations = get_state_estimations(EKF, all_sensor_data)

In [4]:
lidar_xs, lidar_ys = [], []
radar_xs, radar_ys, radar_angles = [], [], []
truth_xs, truth_ys, truth_angles = [], [], []
state_xs, state_ys, state_angles = [], [], []
    
for s, t, p in zip(all_sensor_data, all_ground_truths, all_state_estimations):
    
  if s.get_name() == "lidar":
    x, y = s.get_raw()
    lidar_xs.append(x)
    lidar_ys.append(y)
  else:
    x, y, vx, vy = s.get()
    angle = np.arctan2(vy, vx)
    radar_xs.append(x)
    radar_ys.append(y)
    radar_angles.append(angle)

  x, y, vx, vy = t.get()
  t_angle =  np.arctan2(vy, vx)
  truth_xs.append(x)
  truth_ys.append(y)
  truth_angles.append(t_angle)

  x, y, vx, vy = p.get()
  p_angle =  np.arctan2(vy, vx)
  state_xs.append(x)
  state_ys.append(y)
  state_angles.append(p_angle)

In [5]:
radar_source = ColumnDataSource(data = {
    'x' : radar_xs,
    'y' : radar_ys,
    'angle': radar_angles, 
})

truth_source = ColumnDataSource(data = {
    'x' : truth_xs,
    'y' : truth_ys,
    'angle': truth_angles, 
})

state_source = ColumnDataSource(data = {
    'x' : state_xs,
    'y' : state_ys,
    'angle': truth_angles, 
})

lidar_source = ColumnDataSource(data = {
    'x' : lidar_xs,
    'y' : lidar_ys,
})


hover = HoverTool( tooltips = [
            ("index", "$index"),
            ("x , y", "$x, $y"),
            ("angle radians", "@angle")])


In [6]:
###################################################################
#  RADAR MEASUREMENTS WITH ORIENTATION
###################################################################

hover2 = HoverTool( tooltips = [
            ("index", "$index"),
            ("x , y", "$x, $y"),
            ("angle radians", "@angle")])

p = figure(plot_width = 1000, plot_height = 700, tools = [hover2])

p.triangle(
  'x', 'y', size = 10, 
  fill_color = "firebrick",
  line_color = "orange",
  fill_alpha = 0.2,
  angle = 'angle', 
  line_width = 1, 
  legend = "radar measurements", 
  source = radar_source)


p.cross(
  'x', 'y', size = 5, 
  line_color = "grey",
  angle = 'angle', 
  line_width = 1, 
  legend = "radar measured velocity direction",
  source = radar_source)

p.legend.location = "bottom_right"

show(p)

In [7]:
###################################################################
#  LIDAR MEASUREMENTS
###################################################################

hover3 = HoverTool( tooltips = [
            ("index", "$index"),
            ("x , y", "$x, $y"),
            ("angle radians", "@angle")])

p = figure(plot_width = 1000, plot_height = 700, tools = [hover3])

p.circle(
  'x', 'y', size = 10, 
  fill_color = "navy",
  line_color = "teal",
  fill_alpha = 0.2,
  line_width = 1, 
  legend = "lidar measurements", 
  source = lidar_source)

p.legend.location = "bottom_right"

show(p)

In [8]:
###################################################################
#  GROUND TRUTH WITH ORIENTATION
###################################################################

hover4 = HoverTool( tooltips = [
            ("index", "$index"),
            ("x , y", "$x, $y"),
            ("angle radians", "@angle")])

p = figure(plot_width = 1000, plot_height = 700, tools = [hover4])

p.triangle(
  'x', 'y', size = 15, 
  fill_color = "red",
  line_color = "white",
  fill_alpha = 0.2,
  angle = 'angle', 
  line_width = 1,
  legend = "ground truth",
  source = truth_source)


p.cross(
  'x', 'y', size = 5, 
  line_color = "grey",
  angle = 'angle', 
  line_width = 1, 
  legend = "velocity direction", 
  source = truth_source)

p.legend.location = "bottom_right"
show(p)

In [9]:
###################################################################
#  STATE WITH ORIENTATION
###################################################################

hover5 = HoverTool( tooltips = [
            ("index", "$index"),
            ("x , y", "$x, $y"),
            ("angle radians", "@angle")])

p = figure(plot_width = 1000, plot_height = 700, tools = [hover5])

p.triangle(
  'x', 'y', size = 20, 
  fill_color = "teal",
  line_color = "white",
  fill_alpha = 0.2,
  angle = 'angle', 
  line_width = 1, 
  legend = "state prediction: location", 
  source = state_source)


p.x(
  'x', 'y', size = 5, 
  line_color = "grey",
  angle = 'angle', 
  line_width = 1,
  legend = "state prediction: velocity directions",
  source = state_source)

p.legend.location = "bottom_right"

show(p)

In [10]:
hover6 = HoverTool( tooltips = [
            ("index", "$index"),
            ("x , y", "$x, $y"),
            ("angle radians", "@angle")])

p = figure(plot_width = 1000, plot_height = 700, tools = [hover6])

p.square(
  'x', 'y', size = 3, 
  fill_color = "orange",
  line_color = "orange",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 1, 
  legend = "radar measurements", 
  source = radar_source)

p.circle(
  'x', 'y', size = 15, 
  fill_color = "green",
  line_color = "white",
  fill_alpha = 0.2,
  line_width = 1, 
  legend = "lidar measurements", 
  source = lidar_source)

p.triangle(
  'x', 'y', size = 7, 
  fill_color = "violet",
  line_color = "violet",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 0.5, 
  legend = "ground truth",
  source = truth_source)


p.triangle(
  'x', 'y', size = 2, 
  fill_color = "black",
  line_color = "black",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 1, 
  legend = "state predictions", 
  source = state_source)

p.legend.location = "bottom_right"
show(p)

In [11]:
hover7 = HoverTool( tooltips = [
            ("index", "$index"),
            ("x , y", "$x, $y"),
            ("angle radians", "@angle")])

p = figure(plot_width = 1000, plot_height = 700, tools = [hover7])

p.square(
  'x', 'y', size = 3, 
  fill_color = "violet",
  line_color = "violet",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 1, 
  legend = "radar measurements", 
  source = radar_source)

p.circle(
  'x', 'y', size = 10, 
  fill_color = "green",
  line_color = "white",
  fill_alpha = 0.4,
  line_width = 1, 
  legend = "lidar measurements", 
  source = lidar_source)

p.line(state_xs, state_ys, line_width = 2, color='orange', legend = "state predictions")
p.line(truth_xs, truth_ys, line_dash = "4 4", line_width = 1, color='navy', legend = "ground truth")

p.legend.location = "bottom_right"
show(p)